Poolen
-
Dit jupyter notebook is een uitstapje van het orginele plan. Met dit jupyter notebook kunnen de batches gepooled worden in verschillende formaten. Dit kan gebruikt worden om een afwijking over langere tijd te bekijken of simpelweg om het dataframe te verkleinen.

Deze stap wordt gezet tussen het maken van de dif_batches en de PCA_plotting en is optioneel. Als niet gepooled hoeft te worden kan deze stap overgeslagen worden.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

dif_batches = pd.read_csv(r'C:\Users\pimja\Documents\HU\Q1\PRJ\Code\100_Batches/dif_batches.csv')
dif_batches


,Aeration rate(Fg:L/h),Sugar feed rate(Fs:L/h),Acid flow rate(Fa:L/h),Base flow rate(Fb:L/h),Heating/cooling water flow rate(Fc:L/h),Heating water flow rate(Fh:L/h),Water for injection/dilution(Fw:L/h),Air head pressure(pressure:bar),Dumped broth flow(Fremoved:L/h),Substrate concentration(S:g/L),...,pH(pH:pH),Temperature(T:K),Generated heat(Q:kJ),carbon dioxide percent in off-gas(CO2outgas:%),PAA flow(Fpaa:PAA flow (L/h)),Oil flow(Foil:L/hr),Oxygen Uptake Rate(OUR:(g min^{-1})),Carbon evolution rate(CER:g/h),Ammonia shots(NH3_shots:kgs),Batch_id
0,0.0,0.0,0.000000,16.955571,9.833400,-222.957829,0.0,1.110223e-16,0.0,-0.044131,...,-0.066579,0.547857,-208.977785,0.001140,0.000000,0.0,0.524412,0.000434,0.0,1
1,0.0,0.0,0.000000,28.851774,18.154900,-65.725471,0.0,1.110223e-16,0.0,-0.044696,...,-0.020950,0.286429,-265.539999,0.001561,0.000000,0.0,0.119380,0.000592,0.0,1
2,0.0,0.0,-0.112099,27.324421,9.598100,0.000000,0.0,1.110223e-16,0.0,-0.045464,...,0.024850,0.275714,-260.358285,0.001667,0.000000,0.0,0.020399,0.000633,0.0,1
3,0.0,0.0,-0.074636,12.001143,4.339400,-86.107614,0.0,1.110223e-16,0.0,-0.045443,...,0.053650,-0.040714,-336.679285,0.001677,0.000000,0.0,-0.000965,0.000638,0.0,1
4,0.0,0.0,0.370469,1.117214,-92.986786,-8.610671,0.0,1.110223e-16,0.0,-0.045429,...,0.059336,0.047143,-307.634285,0.001655,0.000000,0.0,-0.003717,0.000628,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85475,0.0,0.0,0.000000,-64.412686,49.482949,-53.926700,0.0,-2.220446e-16,0.0,62.102572,...,0.006379,-0.047143,-54.078571,-0.315271,2.376571,0.0,-0.735050,-0.302007,0.0,100
85476,0.0,0.0,0.000000,-66.351800,-13.442551,52.807975,0.0,-2.220446e-16,0.0,62.219579,...,-0.005350,0.102857,-0.797857,-0.315879,2.370586,0.0,-0.745504,-0.302600,0.0,100
85477,0.0,0.0,0.000000,-63.990186,34.156367,-29.314244,0.0,-2.220446e-16,0.0,62.336588,...,-0.013000,-0.009286,-40.595071,-0.316471,2.371507,0.0,-0.756183,-0.303071,0.0,100
85478,0.0,0.0,0.000000,-58.813786,-12.415199,124.575711,0.0,-2.220446e-16,0.0,62.452593,...,-0.013221,0.336429,77.538000,-0.316957,2.380707,0.0,-0.757057,-0.303636,0.0,100


Voor het poolen van een batch is deze functie geschreven. Asl input voor de functie moet een poolsize en een batch gegeven worden. Hierna wordt de batch vanaf het begin in stukken gehakt ter grote van de batchsize. Als er een stuk overblijft op het einde wordt dit niet gebruikt. Elke pool wordt gesommeerd zodat er één rij overblijft per pool.

Aan het eind worden alle gesommeerde pools in één dataframe gereturned. 

In [2]:
def pool_batches(poolsize, batch):
    if poolsize == 1:
        return batch

    pooled_batches = pd.DataFrame()

    batch_length = len(batch)
    extra_timestamps = batch_length % poolsize

    if extra_timestamps:
        batch = batch.head(batch_length-extra_timestamps)
        batch_length = len(batch)
    
    for i in range(batch_length // poolsize):
        pool = batch.iloc[i*poolsize:(i+1)*poolsize]
        pool = pool.abs().sum()
        pool['Timestamp'] = (i + 1) * poolsize
        pooled_batches = pd.concat([pooled_batches, pool], axis=1)

    return pooled_batches.T

In het volgende stuk code maak ik gebruik van de functie. Met behulp van een for loop stop ik alle batches door de functie. Aan het eind heb ik een dataframe over met alle pools.

In [4]:
pooled_batches = pd.DataFrame()
done = False
POOLSIZE = 20
for i in range(101):
    temp_batch = pool_batches(POOLSIZE,dif_batches[dif_batches["Batch_id"] == i])
    pooled_batches = pd.concat([pooled_batches, temp_batch])
pooled_batches['Batch_id'] = pooled_batches['Batch_id']//POOLSIZE
pooled_batches['Faulty'] = pooled_batches['Batch_id'] > 90
pooled_batches.index = list(range(len(pooled_batches)))
pooled_batches

,Aeration rate(Fg:L/h),Sugar feed rate(Fs:L/h),Acid flow rate(Fa:L/h),Base flow rate(Fb:L/h),Heating/cooling water flow rate(Fc:L/h),Heating water flow rate(Fh:L/h),Water for injection/dilution(Fw:L/h),Air head pressure(pressure:bar),Dumped broth flow(Fremoved:L/h),Substrate concentration(S:g/L),...,Generated heat(Q:kJ),carbon dioxide percent in off-gas(CO2outgas:%),PAA flow(Fpaa:PAA flow (L/h)),Oil flow(Foil:L/hr),Oxygen Uptake Rate(OUR:(g min^{-1})),Carbon evolution rate(CER:g/h),Ammonia shots(NH3_shots:kgs),Batch_id,Timestamp,Faulty
0,0.0,0.0,4.906304,238.571317,500.534273,506.546440,0.0,2.220446e-15,0.0,0.975620,...,5787.495292,0.027719,0.000000,0.0,0.842884,0.010536,0.0,1.0,20.0,False
1,0.0,0.0,0.000000,157.845736,150.154679,4.235639,0.0,2.220446e-15,0.0,1.116864,...,6177.755671,0.025026,0.000000,0.0,0.267870,0.009534,0.0,1.0,40.0,False
2,0.0,0.0,0.444484,241.161600,740.100214,77.168623,0.0,2.220446e-15,0.0,1.496355,...,6500.554242,0.135286,0.000000,0.0,1.424782,0.072152,0.0,1.0,60.0,False
3,0.0,0.0,0.556081,294.064500,1670.234786,491.573895,0.0,2.220446e-16,0.0,3.224266,...,5820.736726,0.129764,0.000000,0.0,1.972338,0.069250,0.0,1.0,80.0,False
4,0.0,0.0,0.111287,750.622643,2333.650357,82.388586,0.0,0.000000e+00,0.0,0.095226,...,5901.126143,0.555931,0.000000,0.0,2.787246,0.296856,0.0,1.0,100.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4267,0.0,0.0,0.000000,1196.689486,507.313802,848.203797,0.0,4.440892e-15,0.0,1050.596005,...,888.675571,5.751943,38.370829,0.0,11.804550,5.514436,0.0,100.0,920.0,True
4268,0.0,0.0,0.000000,1168.727657,518.306181,786.909263,0.0,4.440892e-15,0.0,1091.870130,...,787.526286,5.949650,38.056557,0.0,11.574139,5.703707,0.0,100.0,940.0,True
4269,0.0,0.0,14.340592,906.293870,521.778182,880.711718,0.0,4.440892e-15,0.0,1135.677933,...,703.240857,6.052743,37.325971,0.0,13.667070,5.801200,0.0,100.0,960.0,True
4270,0.0,0.0,19.817549,1217.744737,557.600130,856.184106,0.0,4.440892e-15,0.0,1181.287104,...,835.557429,6.011971,47.212393,0.0,13.447560,5.758729,0.0,100.0,980.0,True


In [ ]:
#pooled_batches.to_csv(r'C:\Users\pimja\Documents\HU\Q1\PRJ\Code\100_Batches/pooled_batches_20.csv', index=False)